In [1]:
#| hide
!pip install -Uqq nixtla
!pip install pandas_market_calendars

In [2]:
#| hide 
from nixtla.utils import in_colab

In [3]:
#| hide 
IN_COLAB = in_colab()

In [4]:
#| hide
if not IN_COLAB:
    from nixtla.utils import colab_badge
    from dotenv import load_dotenv

# Custom frequencies 

When working with time series data, the frequency of the timestamps is a crucial factor that can significantly impact the forecasting results. Regular frequencies like daily, weekly, or monthly are straightforward to handle. However, irregular or custom frquencies can be challenging to work with.

Our forecast method is designed to handle these types of time series data, provided that the frequency of the series is specified correctly. In this tutorial, we will show how to work with custom frequencies, such as the days the U.S. stock market is open.

## 1. Import packages 
First, we import the required packages and initialize the Nixtla client

In [5]:
import pandas as pd
import pandas_market_calendars as mcal
from nixtla import NixtlaClient

In [6]:
nixtla_client = NixtlaClient(
    # defaults to os.environ.get("NIXTLA_API_KEY")
    api_key = 'my_api_key_provided_by_nixtla'
)

> 👍 Use an Azure AI endpoint
> 
> To use an Azure AI endpoint, remember to set also the `base_url` argument:
> 
> `nixtla_client = NixtlaClient(base_url="you azure ai endpoint", api_key="your api_key")`

In [7]:
#| hide
if not IN_COLAB:
    nixtla_client = NixtlaClient()

## 2. Load data

In this example, we will use the daily stock prices of Palantir technologies (PLTR) from 2020 to 2023. The dataset contains information up to 2023-09-22, but here we will remove everything before 2023-08-28 so that we can show how a custom frequency removes the day the stock market close due to the Labor Day holiday.

In [12]:

pltr_df = pd.read_csv('https://raw.githubusercontent.com/Nixtla/transfer-learning-time-series/main/datasets/openbb/pltr.csv', parse_dates=['date'])
pltr_df = pltr_df.query('date < "2023-08-28"')
pltr_df.head()

,date,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits
0,2020-09-30,10.00,11.41,9.11,9.50,9.50,338584400,0.0,0.0
1,2020-10-01,9.69,10.10,9.23,9.46,9.46,124297600,0.0,0.0
2,2020-10-02,9.06,9.28,8.94,9.20,9.20,55018300,0.0,0.0
3,2020-10-05,9.43,9.49,8.92,9.03,9.03,36316900,0.0,0.0
4,2020-10-06,9.04,10.18,8.90,9.90,9.90,90864000,0.0,0.0


## 3. Define a custom frequency 

In [139]:
dates = pd.DatetimeIndex(sorted(pltr_df['date'].unique()))

nyse = mcal.get_calendar('NYSE')
trading_days = nyse.valid_days(start_date=dates.min(), end_date="2025-01-01")

trading_days = trading_days.tz_localize(None)

all_weekdays = pd.date_range(start=dates.min(), end="2025-01-01", freq='B')


holidays = all_weekdays.difference(trading_days)


custom_bday = pd.offsets.CustomBusinessDay(holidays=holidays)

fcst_pltr_df = nixtla_client.forecast(
    df=pltr_df, h=14, freq=custom_bday,
    time_col='date', target_col='Close',
)

INFO:nixtla.nixtla_client:Validating inputs...
/Users/marianamenchero/Documents/nixtla/nixtla/nixtla_client.py:383: UserWarning: `df` contains the following exogenous features: ['Open', 'High', 'Low', 'Adj Close', 'Volume', 'Dividends', 'Stock Splits'], but `X_df` was not provided and they were not declared in `hist_exog_list`. They will be ignored.
  warnings.warn(
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...


In [140]:
fcst_pltr_df

,date,TimeGPT
0,2023-08-28,14.539908
1,2023-08-29,14.439324
2,2023-08-30,14.423326
3,2023-08-31,14.562670
4,2023-09-01,14.672757
5,2023-09-05,14.579653
6,2023-09-06,14.584100
7,2023-09-07,14.475585
8,2023-09-08,14.386601
9,2023-09-11,14.435411
